In [105]:
import pandas as pd
import matplotlib.pyplot as plt

import pyarrow.parquet as pq
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


data = pd.read_csv('/Users/Boaz/Documents/DataMining/prepro_data_RF.csv')
#data = data.drop('time',axis = 1)
#data = data.drop('n_days_id',axis = 1)
data = data.drop('season',axis =1 )
#data = data.drop('id', axis = 1)
data['mood'] = round(data['mood'])
data['circumplex.arousal'] = round(data['circumplex.arousal'])
data['circumplex.valence'] = round(data['circumplex.valence'])
data['target_mood_plus1'] = round(data['target_mood_plus1'])

In [106]:
target = []
for r in data['target_mood_plus1']:
    if r < 5:
        target.append(-1)
    if (r >= 5) & (r <=7):
        target.append(0)
    if r > 7:
        target.append(1)

data['target'] = target

new_mood = []
for r in data['mood']:
    if r < 5:
        new_mood.append(-1)
    if (r >= 5) & (r <=7):
        new_mood.append(0)
    if r > 7:
        new_mood.append(1)

data['target'] = target
data['mood'] = new_mood
# data = data.drop('time',axis = 1)
# data = data.drop('id',axis = 1)
data = data.drop('target_mood_plus1',axis = 1)

In [107]:
data

,id,time,mood,circumplex.arousal,circumplex.valence,activity,screen,sms,appCat.builtin,appCat.communication,...,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,summer,spring,winter,target
0,AS14.01,2014-02-26,0,-0.0,1.0,0.000000,0.000000,2.0,0.000,0.000,...,0.000,0.0000,0.000,0.0,0.000,0.000,0,0,1,0
1,AS14.01,2014-03-21,0,0.0,0.0,3.083152,9100.601001,0.0,1255.653,5903.482,...,139.195,3945.8130,915.445,0.0,598.754,0.000,0,1,0,0
2,AS14.01,2014-03-22,0,1.0,0.0,3.790084,6142.161000,1.0,480.124,4962.918,...,98.143,439.6320,37.305,0.0,117.621,0.000,0,1,0,0
3,AS14.01,2014-03-23,0,0.0,1.0,2.141117,6033.096002,0.0,615.324,4495.049,...,72.823,900.8390,0.000,0.0,30.086,30.386,0,1,0,0
4,AS14.01,2014-03-24,0,1.0,0.0,1.895065,10422.503001,0.0,816.953,6809.575,...,66.558,2644.4390,54.701,0.0,178.732,0.000,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,AS14.33,2014-05-25,0,-1.0,0.0,3.713814,6659.054999,1.0,1282.030,1567.344,...,42.362,2868.6710,0.000,0.0,96.982,0.000,1,0,0,0
1207,AS14.33,2014-05-26,0,-0.0,0.0,1.497619,5975.597999,3.0,846.084,1235.537,...,151.582,5441.2640,0.000,0.0,0.000,0.000,1,0,0,0
1208,AS14.33,2014-05-27,0,-1.0,0.0,0.304902,3377.526001,2.0,365.069,2078.403,...,142.686,2010.3640,0.000,0.0,56.173,0.000,1,0,0,1
1209,AS14.33,2014-05-28,1,0.0,1.0,2.479228,6241.620998,1.0,1751.404,1936.415,...,336.913,4487.4515,0.000,0.0,30.666,0.000,1,0,0,0


In [108]:
# d = {}
# ids = data['id'].unique()
# for id in ids:
#     data_per_id = data.loc[(data['id'] == id)]
#     data_time = data_per_id.set_index(['time'])
#     data_time.index = pd.to_datetime(data_time.index)
#     data_arima = data_time.sort_index()
#     d["{0}".format(id)] = data_arima

In [109]:
# data = data.set_index(['time'])
# data.index = pd.to_datetime(data.index)
# data

In [112]:
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

me = []
mae = []
mpa = []

for user in d.keys():
    data_per_id = d[user]
    labels = np.array(data_per_id['target'])
    features= data_per_id.drop('target', axis = 1)
    features = np.array(features)
    split = int(0.75 * (len(data_per_id)))
    X_train = data_per_id.target[:split]
    #y_train = data_per_id.target[:split]
    y_test = data_per_id.target[split:]


    model = pm.auto_arima(X_train, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

    #print(model.summary())
    n_periods = len(X_test)
    fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
    fc = fc.round()
    me.append(np.mean(fc - y_test))             # ME
    mae.append(np.mean(np.abs(fc - y_test)))    # MAE
    mpa.append(np.mean(np.abs(fc - y_test))/len(y_test)) # MAPE
    
    

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.08 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=50.451, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=42.007, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=48.451, Time=0.01 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=43.399, Time=0.07 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.12 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=40.062, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=41.469, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=inf, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=inf, Time=0.03 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=inf, Time=0.07 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0]          
Total fit time: 0.580 seconds


C:\Users\Boaz\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=39.152, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=51.570, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=45.819, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=49.681, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=39.386, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.08 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=37.940, Time=0.06 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=38.830, Time=0.06 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.16 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=35.959, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=36.221, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=37.156, Time=0.05 sec
 ARIMA(0,1,3)(0,0,0)[0]             : AIC=36.851, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=37.377, Time=0.03 sec
 ARIMA

C:\Users\Boaz\anaconda3\lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=78.226, Time=0.01 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=72.188, Time=0.03 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=72.697, Time=0.03 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.10 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.11 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=70.322, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=70.579, Time=0.01 sec
 ARIMA(3,1,0)(0,0,0)[0]             : AIC=70.870, Time=0.02 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=66.844, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=64.982, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=63.036, Time=0.02 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=64.990, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=inf, Time=0.11 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 0.739 seconds
Performing stepwise search to minimize 

...

C:\Users\Boaz\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.10 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=49.464, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=inf, Time=0.06 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=inf, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=54.277, Time=0.01 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=45.552, Time=0.04 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=47.304, Time=0.04 sec
 ARIMA(1,1,3)(0,0,0)[0]             : AIC=47.446, Time=0.06 sec
 ARIMA(0,1,3)(0,0,0)[0]             : AIC=45.447, Time=0.03 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.09 sec

Best model:  ARIMA(0,1,3)(0,0,0)[0]          
Total fit time: 0.748 seconds
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=inf, Time=0.06 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=54.378, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=55.417, Time=0.01 sec
 A

C:\Users\Boaz\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


 ARIMA(1,0,0)(0,0,0)[0]             : AIC=34.198, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=34.144, Time=0.01 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=29.914, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=31.881, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=31.859, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.07 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0] intercept
Total fit time: 0.196 seconds


C:\Users\Boaz\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


In [122]:
np.nanmean(mpa)

0.028269290427307848

In [94]:
train = data.target[:951]
test = data.target[951:]
print(len(test))
# forecasting 317 points

260


In [ ]:
#!pip install pmdarima 

In [95]:
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

model = pm.auto_arima(train, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=978.046, Time=0.12 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=1218.025, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=1076.052, Time=0.04 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=1122.633, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=974.390, Time=0.15 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=1033.333, Time=0.06 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=974.471, Time=0.26 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=981.927, Time=0.24 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=975.083, Time=0.17 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=1024.071, Time=0.09 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=978.361, Time=0.34 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=957.023, Time=0.42 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=956.954, Time=0.34 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=998.549, Time=0.13 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=

In [97]:
n_periods = 260
fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
fc.values

array([0.30598729, 0.28883299, 0.27420393, 0.26172838, 0.25108933,
       0.24201642, 0.23427912, 0.2276808 , 0.2220538 , 0.21725514,
       0.21316288, 0.20967303, 0.20669692, 0.2041589 , 0.20199451,
       0.20014873, 0.19857466, 0.19723231, 0.19608756, 0.19511132,
       0.1942788 , 0.19356883, 0.19296337, 0.19244704, 0.19200672,
       0.19163121, 0.19131099, 0.1910379 , 0.19080501, 0.19060641,
       0.19043704, 0.19029261, 0.19016943, 0.19006439, 0.18997481,
       0.18989842, 0.18983328, 0.18977772, 0.18973034, 0.18968994,
       0.18965548, 0.1896261 , 0.18960104, 0.18957967, 0.18956145,
       0.1895459 , 0.18953265, 0.18952135, 0.18951171, 0.18950349,
       0.18949648, 0.1894905 , 0.18948541, 0.18948106, 0.18947735,
       0.18947419, 0.18947149, 0.18946919, 0.18946723, 0.18946556,
       0.18946413, 0.18946292, 0.18946188, 0.189461  , 0.18946024,
       0.1894596 , 0.18945905, 0.18945858, 0.18945818, 0.18945784,
       0.18945755, 0.18945731, 0.1894571 , 0.18945692, 0.18945

In [98]:
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    #acf1 = acf(fc-test)[1]                      # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(fc, test.values)

{'mape': inf,
 'me': -0.27673022951208454,
 'mae': 0.4792525365961827,
 'mpe': inf,
 'rmse': 0.5692017911358612,
 'corr': 0.13540834353597087,
 'minmax': 0.9087606138233644}

In [102]:
# Build Model
model = ARIMA(train, order=(3, 1, 3))  
fitted = model.fit(disp=-1)  

# Forecast
fc, se, conf = fitted.forecast(317, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

NotImplementedError: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been removed in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and statsmodels.tsa.SARIMAX.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained. It also offers alternative specialized
parameter estimators.


C:\Users\Boaz\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
C:\Users\Boaz\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """


{'mape': inf,
 'me': -0.11343772883599001,
 'mae': 0.475417298882478,
 'mpe': inf,
 'rmse': 0.5086121428236583,
 'corr': -0.0002364334775461686,
 'minmax': 0.8622508577078145}